# **0 - Data Wrangling: Balearic Grids**

##### Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import copy
import leiap_survey.utils as utils

from shapely.geometry import Point
from pathlib import Path

# Read data

##### Project main path

In [2]:
# Change path so that it points to  were data is located
pth = Path(r'../data')
pth 

WindowsPath('../data')

### Read Geographic information 

##### Read grid 80 x80 m (polygons)

In [3]:
fn = pth / 'geography' / 'grid80.shp'
fn

WindowsPath('../data/geography/grid80.shp')

In [4]:
grid = gpd.read_file(fn, encoding= 'utf-8')
grid = grid.reset_index()
del grid['Id']
grid.head(2)

,index,geometry
0,0,"POLYGON ((531966.615 4385295.031, 531966.615 4..."
1,1,"POLYGON ((532046.615 4385295.031, 532046.615 4..."


Recover projection

In [5]:
proj = grid.crs. to_epsg()
proj

25831

### Read Balearic survey data
This file contains the name of each pottery production, its time interval and possible uses 

In [6]:
fn = pth / 'artifacts' / 'by_pts' / 'balearic_presence_types.txt'
pts_pres_types = pd.read_csv(fn, index_col= ['SurveyPointId', 'Easting', 'Northing'])
pts_pres_types.head(2)

,,,Amphora,Coarseware,Commonware,Fineware
SurveyPointId,Easting,Northing,,,,
02001d-77-012,531989.73,4385614.15,1.0,0.0,0.0,0.0
02001d-80-008,531872.15,4385569.72,1.0,0.0,0.0,0.0


In [7]:
fn = pth / 'artifacts' / 'by_pts' / 'balearic_presence_productions.txt'
pts_pres_prod = pd.read_csv(fn, index_col= ['SurveyPointId', 'Easting', 'Northing'])
pts_pres_prod.head(2)

,,,AtBlckGlz,Balearic,CamBlckGlzA,CamBlckGlzB,CmedPuAm,EbBlckGlz,EbPuAm,EbPuCom,GrItAm,IbAm,IbCom,ItAm,ItCom,MasAm,PuAm,PuCom,SoItVesCaAm
SurveyPointId,Easting,Northing,,,,,,,,,,,,,,,,,
02001d-77-012,531989.73,4385614.15,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02001d-80-008,531872.15,4385569.72,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Spatial Joins with grids

#### Convert artifacts into geopandas

In [8]:
# What dataset? Choose either pottery types (pts_pres_types)  or originals (pts_pres_prod)
ds = pts_pres_types
pts = ds.reset_index().copy()

In [9]:
# Generate geopandas point dataframe
pts['geometry'] = pts.apply(lambda pt: Point(pt.Easting, pt.Northing), axis = 1)
pts = gpd.GeoDataFrame(pts)

# Project to the same projection than grid
pts = pts.set_crs(epsg= proj)

#### Spatial join with the grid

Generate a spatial join where each point will be associated with the grid id that contains it

In [10]:
pts= (pts.sjoin(grid, how='inner')
         .drop(columns = 'index')
         .rename(columns = {'index_right': 'grid80'})
)
pts = pts.astype({'grid80': 'int64'})
pts.head(3)

,SurveyPointId,Easting,Northing,Amphora,Coarseware,Commonware,Fineware,geometry,grid80
0,02001d-77-012,531989.73,4385614.15,1.0,0.0,0.0,0.0,POINT (531989.730 4385614.150),13
1,02001d-80-008,531872.15,4385569.72,1.0,0.0,0.0,0.0,POINT (531872.150 4385569.720),11
2,02001d-82-024,532116.88,4385546.76,1.0,0.0,0.0,0.0,POINT (532116.880 4385546.760),14


In [11]:
# Drop geometry as we do not need it anymore
del pts['geometry']

In [12]:
# group by grid id
grid = pts.groupby('grid80').sum()
grid.head(3)

,Easting,Northing,Amphora,Coarseware,Commonware,Fineware
grid80,,,,,,
2,1596442.99,13156043.68,2.0,1.0,0.0,0.0
11,531872.15,4385569.72,1.0,0.0,0.0,0.0
13,531989.73,4385614.15,1.0,0.0,0.0,0.0


In [13]:
# presence or absence
grid = grid.where(grid<1.0, 1)

In [33]:
# Choose '_types.txt' or '_productions.txt' depending on the dataset used in spatial join
fn = pth / 'artifacts' / 'grid80_types.txt'

# Un comment to save
#grid.to_csv(fn)